In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
def save_pkl(file_dir, data):
    f = open(file_dir,"wb")
    pickle.dump(data, f, protocol=4)
    f.close()
    
def read_pkl(file_dir):
    f = open(file_dir,"rb")
    data = pickle.load(f)
    return data

In [3]:
Grid = pd.read_csv("London/London_historical_meo_grid.csv")
Grid.head(3)

,stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed/kph
0,london_grid_000,-2.0,50.5,2017-01-01 00:00:00,9.36,1024.81,77.90,250.88,23.74
1,london_grid_001,-2.0,50.6,2017-01-01 00:00:00,9.09,1024.25,78.96,249.86,21.81
2,london_grid_002,-2.0,50.7,2017-01-01 00:00:00,8.30,1022.55,82.13,245.35,16.08


In [4]:
Stations = pd.read_csv("London/London_AirQuality_Stations.csv", index_col=0)
Stations.sort_values(by=['Latitude'])

,api_data,need_prediction,historical_data,Latitude,Longitude,SiteType,SiteName
ST5,True,True,True,51.389287,-0.141662,Industrial,Sutton - Beddington Lane north
CR8,NaN,NaN,True,51.410039,-0.127523,Urban Background,Croydon - Norbury Manor
TD5,True,NaN,True,51.425256,-0.345608,Suburban,Richmond Upon Thames - Bushy Park
GR4,True,True,True,51.452580,0.070766,Suburban,Greenwich - Eltham
GB0,NaN,NaN,True,51.456300,0.085606,Roadside,Greenwich and Bexley - Falconwood FDMS
GR9,True,True,True,51.456357,0.040725,Roadside,Greenwich - Westhorne Avenue
BX9,True,NaN,True,51.465983,0.184877,Suburban,Bexley - Slade Green FDMS
BX1,True,NaN,True,51.465983,0.184877,Suburban,Bexley - Slade Green
LW2,True,True,True,51.474954,-0.039641,Roadside,Lewisham - New Cross
GN3,True,True,True,51.486957,0.095111,Roadside,Greenwich - Plumstead High Street


In [5]:
# The purpose of this part is to find the neighbour stations of each station

# import math
station_position = {}
station_name = Stations.index.values
station_Latitude = np.asarray(Stations['Latitude'].values)
station_Longitude = np.asarray(Stations['Longitude'].values)
for i in range(len(station_name)):  
    name = station_name[i]
    if name!= 'KC1' and name!= 'BX9' and name!= 'CT2':
        point_list = []
        point_dist = []
        temp_name_0 = []
        temp_dist_0 = []
        temp_name_1 = []
        temp_dist_1 = []
        temp_name_2 = []
        temp_dist_2 = []
        temp_name_3 = []
        temp_dist_3 = []
        # angle_list = []
        dist_Latitude = station_Latitude - station_Latitude[i]
        dist_Longitude = station_Longitude - station_Longitude[i]
        dist_list = np.sqrt(np.square(dist_Latitude) + np.square(dist_Longitude))
        for j in range(len(station_name)):
            if j!=i and station_name[j] != 'BX9' and station_name[j] != 'KC1' and station_name[j] != 'CT2':
                # if  dist_list[j]< 0.2:
                    # tan_list.append(dist_Latitude[j]/dist_Longitude[j])
                    lat = dist_Latitude[j]
                    lon = dist_Longitude[j]
                    if(lat>=0 and lon>=0):
                        temp_name_0.append(station_name[j])
                        temp_dist_0.append(dist_list[j])
                    elif(lat>=0 and lon<0):
                        temp_name_1.append(station_name[j])
                        temp_dist_1.append(dist_list[j])      
                    elif(lat<0 and lon<0):
                        temp_name_2.append(station_name[j])
                        temp_dist_2.append(dist_list[j])
                    elif(lat<0 and lon>=0):
                        temp_name_3.append(station_name[j])
                        temp_dist_3.append(dist_list[j])
                    # angle_list.append(math.atan(dist_Latitude[j]/dist_Longitude[j]))
                    # print(station_name[j],dist_list[j],math.atan(dist_Latitude[j]/dist_Longitude[j]))
        if(len(temp_name_0)):
            j = temp_dist_0.index(min(temp_dist_0))
            point_list.append(temp_name_0[j])
            point_dist.append(temp_dist_0[j])
        if(len(temp_name_1)):
            j = temp_dist_1.index(min(temp_dist_1))
            point_list.append(temp_name_1[j])
            point_dist.append(temp_dist_1[j])
        if(len(temp_name_2)):
            j = temp_dist_2.index(min(temp_dist_2))
            point_list.append(temp_name_2[j])
            point_dist.append(temp_dist_2[j])
        if(len(temp_name_3)):
            j = temp_dist_3.index(min(temp_dist_3))
            point_list.append(temp_name_3[j])
            point_dist.append(temp_dist_3[j])
        print(name, 'is', len(point_list))
        # angle_list = np.arctan(tan_list)
        # print(angle_list)
        print(point_list)
        print(point_dist)
        station_position[name] = [station_Latitude[i],station_Longitude[i],point_list,point_dist]

BX1 is 3
['HV1', 'GN3', 'GB0']
[0.05854214692001993, 0.09218381043704524, 0.09974227977013109]
BL0 is 4
['RB7', 'CD9', 'KF1', 'CT3']
[0.21402438249322162, 0.006296476795510455, 0.08765291493385774, 0.04881741355178004]
CD9 is 4
['RB7', 'CD1', 'MY7', 'BL0']
[0.21603866996190044, 0.049091191670160975, 0.026054210045526143, 0.006296476795510455]
CD1 is 4
['RB7', 'HR1', 'KF1', 'MY7']
[0.259424699407525, 0.14350890824266174, 0.04468573745975535, 0.029970329944796262]
CT3 is 4
['TH4', 'BL0', 'CR8', 'LW2']
[0.06935755329610337, 0.04881741355178004, 0.11511702276146805, 0.054462563547024695]
CR8 is 3
['LW2', 'MY7', 'ST5']
[0.10925750843305446, 0.11571126777457238, 0.025110705756424743]
GN0 is 4
['RB7', 'TH4', 'GR4', 'GN3']
[0.07945287966747655, 0.0859898077631167, 0.038089794866871085, 0.02140860315387216]
GR4 is 3
['GB0', 'GR9', 'CR8']
[0.01529915030320342, 0.03027750666749245, 0.20280104586022082]
GN3 is 4
['HV1', 'GN0', 'GB0', 'BX1']
[0.11541905231848792, 0.02140860315387216, 0.032096676993

In [6]:
# print(station_position)
for name in station_position.keys():
    print(name)
    print(station_position[name])

BX1
[51.46598327, 0.184877127, ['HV1', 'GN3', 'GB0'], [0.05854214692001993, 0.09218381043704524, 0.09974227977013109]]
BL0
[51.522287, -0.12584800000000002, ['RB7', 'CD9', 'KF1', 'CT3'], [0.21402438249322162, 0.006296476795510455, 0.08765291493385774, 0.04881741355178004]]
CD9
[51.52770662, -0.129053205, ['RB7', 'CD1', 'MY7', 'BL0'], [0.21603866996190044, 0.049091191670160975, 0.026054210045526143, 0.006296476795510455]]
CD1
[51.544219, -0.175284, ['RB7', 'HR1', 'KF1', 'MY7'], [0.259424699407525, 0.14350890824266174, 0.04468573745975535, 0.029970329944796262]]
CT3
[51.51384718, -0.077765682, ['TH4', 'BL0', 'CR8', 'LW2'], [0.06935755329610337, 0.04881741355178004, 0.11511702276146805, 0.054462563547024695]]
CR8
[51.410039000000005, -0.127523, ['LW2', 'MY7', 'ST5'], [0.10925750843305446, 0.11571126777457238, 0.025110705756424743]]
GN0
[51.490532, 0.074003, ['RB7', 'TH4', 'GR4', 'GN3'], [0.07945287966747655, 0.0859898077631167, 0.038089794866871085, 0.02140860315387216]]
GR4
[51.45258, 0.

In [7]:
save_pkl("London/station_processing.pkl", station_position)

In [8]:
station_position = read_pkl("London/station_processing.pkl")

In [9]:
# the NaN situation of the first original air quality csv file provided by the KDD website

Airs = pd.read_csv("London/London_historical_aqi_forecast_stations_20180331.csv")
missing_values_count = Airs.isnull().sum()
missing_values_count

Unnamed: 0                0
MeasurementDateGMT        0
station_id                0
PM2.5 (ug/m3)         18676
PM10 (ug/m3)          14553
NO2 (ug/m3)           25445
dtype: int64

In [10]:
Airs.head(3)

,Unnamed: 0,MeasurementDateGMT,station_id,PM2.5 (ug/m3),PM10 (ug/m3),NO2 (ug/m3)
0,0,2017/1/1 0:00,CD1,40.0,44.4,36.6
1,1,2017/1/1 1:00,CD1,31.6,34.4,46.2
2,2,2017/1/1 2:00,CD1,24.7,28.1,38.3


In [11]:
# the NaN situation of the second original air quality csv file provided by the KDD website

Airs = pd.read_csv("London/London_historical_aqi_other_stations_20180331.csv")
missing_values_count = Airs.isnull().sum()
missing_values_count

/home/liu/anaconda3/envs/img/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Station_ID             22959
MeasurementDateGMT     22958
PM2.5 (ug/m3)          53621
PM10 (ug/m3)          102438
NO2 (ug/m3)            91703
Unnamed: 5            141633
Unnamed: 6            141633
dtype: int64

In [12]:
# The purpose of this part is to merge two original air quality csv files provided by the KDD website

df = pd.read_csv("London/London_historical_aqi_forecast_stations_20180331.csv")
df.rename(columns={'NO2 (ug/m3)': 'NO2_Concentration',
                   'PM10 (ug/m3)': 'PM10_Concentration',
                   'PM2.5 (ug/m3)': 'PM25_Concentration',
                   'MeasurementDateGMT': 'time'
                  }, inplace=True)
group_ = df.groupby("station_id")
for name, g in group_:
    print(name)

df1 = pd.read_csv("London/London_historical_aqi_other_stations_20180331.csv")
df1.rename(columns={'NO2 (ug/m3)': 'NO2_Concentration',
                    'PM10 (ug/m3)': 'PM10_Concentration',
                    'PM2.5 (ug/m3)': 'PM25_Concentration',
                    'MeasurementDateGMT': 'time',
                    'Station_ID': 'station_id'
                   }, inplace=True)
df = pd.concat([df, df1])

df['time'] = pd.to_datetime(df['time'])
df.index = df['time']
df['time_week'] = df.index.map(lambda x: x.weekday)
df['time_year'] = df.index.map(lambda x: x.year)
df['time_month'] = df.index.map(lambda x: x.month)
df['time_day'] = df.index.map(lambda x: x.day)
df['time_hour'] = df.index.map(lambda x: x.hour)
Airs = df[["station_id", "PM25_Concentration", "PM10_Concentration", "NO2_Concentration",
           'time_year', 'time_month', 'time_week', 'time_day', 'time_hour']]

Airs = Airs.sort_index()
group = Airs.groupby("station_id")
# print(group)
station_group = {}
for name, g in group:
    if name != 'CT2':
        station_group[name] = g.sort_index()[:10897]
#         print(name)
#         print(len(station_group[name]))

BL0
CD1
CD9
GN0
GN3
GR4
GR9
HV1
KF1
LW2
MY7
ST5
TH4


/home/liu/anaconda3/envs/img/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [13]:
# display the negative value count
# padding the negative value into zero
attr_need = ["PM25_Concentration", "PM10_Concentration", "NO2_Concentration"]
for name in station_group:
    print(name)
    df = station_group[name][attr_need]
    print(sum(n < 0 for n in df.values.flatten()))
    station_group[name][attr_need] = df.clip(lower=0)
    df = station_group[name][attr_need]
    print(sum(n < 0 for n in df.values.flatten()))

BL0
2
0
BX1
100
0
BX9
67
0
CD1
27
0
CD9
68
0
CR8
3
0
CT3
107
0
GB0
15
0
GN0
202
0
GN3
59
0
GR4
71
0
GR9
292
0
HR1
0
0
HV1
316
0
KC1
64
0
KF1
0
0
LH0
0
0
LW2
41
0
MY7
8
0
RB7
31
0
ST5
54
0
TD5
0
0
TH4
73
0


In [14]:
# The purpose of this part is to merge two duplicated station pairs

def duplicate_padding(station_group, attr_need, remove_station, keep_station):
    keep_ = station_group[keep_station][attr_need]
    keep_values = keep_.values
    remove_ = station_group[remove_station][attr_need]
    remove_values = remove_.values
    print (keep_values.shape, remove_values.shape)
    print(keep_.isnull().sum())
    print(remove_.isnull().sum())
    for i in range(remove_values.shape[0]):
        for j in range(remove_values.shape[1]):
            if np.isnan(keep_values[i, j]):
                keep_values[i, j] = remove_values[i, j]
    station_group[keep_station].loc[:, attr_need] = keep_values
    keep_ = station_group[keep_station][attr_need]
    print(keep_.isnull().sum())
    print(len(station_group))
    station_group.pop(remove_station)
    print(len(station_group))
    
attr_need = ["PM25_Concentration", "PM10_Concentration", "NO2_Concentration"]
duplicate_padding(station_group, attr_need, "KC1", "KF1")
duplicate_padding(station_group, attr_need, "BX9", "BX1")

(10897, 3) (10897, 3)
PM25_Concentration     4604
PM10_Concentration     4604
NO2_Concentration     10897
dtype: int64
PM25_Concentration     1906
PM10_Concentration    10643
NO2_Concentration        80
dtype: int64
PM25_Concentration     694
PM10_Concentration    4350
NO2_Concentration       80
dtype: int64
23
22
(10897, 3) (10897, 3)
PM25_Concentration    6706
PM10_Concentration      65
NO2_Concentration      444
dtype: int64
PM25_Concentration      262
PM10_Concentration    10897
NO2_Concentration     10897
dtype: int64
PM25_Concentration    256
PM10_Concentration     65
NO2_Concentration     444
dtype: int64
22
21


In [15]:
# padding the NaN value with neighbour values and historical values

def neighbour_padding(station_group, attr_need, station_position):
    normal_change_health = []# the change% when the previous value is smaller than 20
    normal_change_else = []# the change% when the previous value is bigger than 20
    for j in range(len(attr_need)):# calculate the change% in the normal situation 
        change_list_health = []
        change_list_else = []
        for name in station_position.keys():
            keep_values = station_group[name][attr_need].values
            pre_nan = True
            for i in range(keep_values.shape[0]):
                if np.isnan(keep_values[i, j]):
                    this_nan = True
                else:
                    if(not pre_nan):
                        pre_value = keep_values[i-1, j]
                        this_value = keep_values[i, j]
                        if(pre_value>0 and this_value>0):
                            change_percent = (this_value - pre_value) / pre_value
                            if(change_percent<0):
                                change_percent = - change_percent
                            if(pre_value<20):
                                change_list_health.append(change_percent)
                            else:
                                change_list_else.append(change_percent)
                    this_nan = False
                pre_nan = this_nan
        total_number = len(change_list_health)
        average = sum(change_list_health)/total_number
        normal_number = sum(i < average*2  for i in change_list_health)
        # print(attr_need[j],total_number,max(change_list),min(change_list),normal_number)
        print(average,normal_number/total_number)
        normal_change_health.append(normal_number*2)    
        
        total_number = len(change_list_else)
        average = sum(change_list_else)/total_number
        normal_number = sum(i < average*2  for i in change_list_else)
        # print(attr_need[j],total_number,max(change_list),min(change_list),normal_number)
        print(average,normal_number/total_number)
        normal_change_else.append(normal_number*2) 
    
    for name in station_position.keys():# padding the nan value
        print(name)
        neighbour_list = station_position[name][2]
        neighbour_dist = station_position[name][3]
        # print(neighbour_list)
        # print(neighbour_dist)
        
        keep_ = station_group[name][attr_need]
        print(keep_.isnull().sum())
        keep_values = keep_.values           
        neighbour_values_list = []
        for neighbour in neighbour_list:
            neighbour_values_list.append(station_group[neighbour][attr_need].values)

        for j in range(keep_values.shape[1]):
            pre_nan = True
            for i in range(keep_values.shape[0]):
                if np.isnan(keep_values[i, j]):# this value is empty
                    this_nan = True
                    temp_list = []
                    temp_dist = []
                    for k in range(len(neighbour_values_list)):# find neighbour values which are no empty
                        if(not np.isnan(neighbour_values_list[k][i, j])):
                            temp_list.append(neighbour_values_list[k][i, j])
                            temp_dist.append(neighbour_dist[k])
                    if(len(temp_list)):# if exists neighbour values which are no empty
                        temp_normal = min(temp_dist) + max(temp_dist)
                        temp_result = 0
                        temp_ratio = 0
                        for k in range(len(temp_list)): # for each neighbour value
                            # print(temp_list[k])
                            temp_result = temp_result + temp_list[k] * (temp_normal - temp_dist[k])
                            temp_ratio = temp_ratio + (temp_normal - temp_dist[k])
                            # print('is',temp_result/temp_ratio)
                        temp_result = temp_result/temp_ratio # calculate the weighted average result, the weights are related to the distance of each point
                        
                        if(pre_nan):# if the previous value is empty
                            keep_values[i, j] = temp_result
                        else:# if the previous value exists
                            pre_value = keep_values[i-1, j]
                            if(pre_value>0 and temp_result>0):# calculate the change% between the previous value and the weighted average result
                                change_percent = (temp_result - pre_value) / pre_value
                                if(change_percent<0):# if the previous value bigger than the weighted average result
                                    change_percent = - change_percent
                                    if((pre_value<20 and change_percent < normal_change_health[j])
                                      or(pre_value>=20 and change_percent < normal_change_else[j])):
                                        keep_values[i, j] = temp_result
                                    else:# if the weighted average result is smaller than the normal situation
                                        if(pre_value<20):
                                            keep_values[i, j] = pre_value * (1 - normal_change_health[j]) 
                                        else:
                                            keep_values[i, j] = pre_value * (1 - normal_change_else[j]) 
                                else:# if the previous value smaller than the weighted average result
                                    if((pre_value<20 and change_percent < normal_change_health[j])
                                      or(pre_value>=20 and change_percent < normal_change_else[j])):
                                        keep_values[i, j] = temp_result
                                    else:# if the weighted average result is bigger than the normal situation
                                        if(pre_value<20):
                                            keep_values[i, j] = pre_value * (1 + normal_change_health[j]) 
                                        else:
                                            keep_values[i, j] = pre_value * (1 + normal_change_else[j])   
                            else:
                                keep_values[i, j] = temp_result
                    else:# if all of the neighbour values are no empty
                        pass         
                else:# this value is no empty
                    this_nan = False
                pre_nan = this_nan
        station_group[name].loc[:, attr_need] = keep_values
        keep_ = station_group[name][attr_need]
        print(keep_.isnull().sum())
    
attr_need = ["PM25_Concentration", "PM10_Concentration", "NO2_Concentration"]
neighbour_padding(station_group, attr_need, station_position)

0.35897586634299006 0.9199390060289209
0.1405961378432799 0.87744542754799
0.3002761220654578 0.912440613748847
0.1578368191346502 0.8819678758571362
0.27888462410942555 0.8915316503391107
0.16369228051884563 0.8821288442440128
BX1
PM25_Concentration    256
PM10_Concentration     65
NO2_Concentration     444
dtype: int64
PM25_Concentration    0
PM10_Concentration    1
NO2_Concentration     0
dtype: int64
BL0
PM25_Concentration    651
PM10_Concentration    559
NO2_Concentration     148
dtype: int64
PM25_Concentration    1
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CD9
PM25_Concentration    1152
PM10_Concentration    1143
NO2_Concentration       58
dtype: int64
PM25_Concentration    1
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CD1
PM25_Concentration    294
PM10_Concentration    739
NO2_Concentration     921
dtype: int64
PM25_Concentration    1
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CT3
PM25_Concentration    3074
PM10_Concentration

In [16]:
# padding the NaN value with the average value of the previous value and the next value

def time_padding(station_group, attr_need):
    for name in station_position.keys():# padding the nan value
        print(name)
        keep_ = station_group[name][attr_need]
        print(keep_.isnull().sum())
        keep_values = keep_.values           
        
        for j in range(keep_values.shape[1]):
            pre_nan = True
            for i in range(keep_values.shape[0]):
                if np.isnan(keep_values[i, j]):# this value is empty
                    this_nan = True
                    if(i+1 < keep_values.shape[0]):
                        if(not np.isnan(keep_values[i+1, j])):# next value exists
                            if(not pre_nan):# previous value  exists
                                temp_result = (keep_values[i+1, j] + keep_values[i-1, j]) / 2
                            else:# no previous value
                                temp_result = keep_values[i+1, j]
                            keep_values[i, j] = temp_result
                    else:# no next value
                        if(not pre_nan):# previous value  exists
                            keep_values[i, j] = keep_values[i-1, j]
                        
                else:# this value is no empty
                    this_nan = False
                pre_nan = this_nan
                
        station_group[name].loc[:, attr_need] = keep_values
        keep_ = station_group[name][attr_need]
        print(keep_.isnull().sum())
    
attr_need = ["PM25_Concentration", "PM10_Concentration", "NO2_Concentration"]
time_padding(station_group, attr_need)

BX1
PM25_Concentration    0
PM10_Concentration    1
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
BL0
PM25_Concentration    1
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CD9
PM25_Concentration    1
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CD1
PM25_Concentration    1
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CT3
PM25_Concentration    1
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CR8
PM25_Concentration     1
PM10_Concentration    19
NO2_Concentration      7
dtype: int64
PM25_Concentration     0
PM10_Concentr

In [17]:
# second time of padding the NaN value with neighbour values and historical values

attr_need = ["PM25_Concentration", "PM10_Concentration", "NO2_Concentration"]
neighbour_padding(station_group, attr_need, station_position)

0.3379619317999025 0.9184793851343825
0.13790324631052367 0.8773067197503631
0.2702583017839098 0.9078822747770035
0.15249123103009746 0.881590239602904
0.27504792248960663 0.8915259368902644
0.1613784511819762 0.8822659807226065
BX1
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
BL0
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CD9
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CD1
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
dtype: int64
CT3
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     

In [18]:
save_pkl("London/air_processing.pkl", station_group)

In [19]:
station_group_test = read_pkl("London/air_processing.pkl")
station_group['ST5']

,station_id,PM25_Concentration,PM10_Concentration,NO2_Concentration,time_year,time_month,time_week,time_day,time_hour
time,,,,,,,,,
2017-01-01 00:00:00,ST5,25.0,29.0,14.5,2017.0,1.0,6.0,1.0,0.0
2017-01-01 01:00:00,ST5,21.0,25.0,24.0,2017.0,1.0,6.0,1.0,1.0
2017-01-01 02:00:00,ST5,20.0,23.0,28.1,2017.0,1.0,6.0,1.0,2.0
2017-01-01 03:00:00,ST5,22.0,21.0,20.6,2017.0,1.0,6.0,1.0,3.0
2017-01-01 04:00:00,ST5,22.0,26.0,34.6,2017.0,1.0,6.0,1.0,4.0
2017-01-01 05:00:00,ST5,24.0,29.0,23.8,2017.0,1.0,6.0,1.0,5.0
2017-01-01 06:00:00,ST5,16.0,23.0,17.2,2017.0,1.0,6.0,1.0,6.0
2017-01-01 07:00:00,ST5,20.0,27.0,15.2,2017.0,1.0,6.0,1.0,7.0
2017-01-01 08:00:00,ST5,20.0,25.0,20.6,2017.0,1.0,6.0,1.0,8.0


In [20]:
# Check if all values exist
for name in station_group_test:
    print(station_group_test[name].isnull().sum())

station_id            0
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
time_year             0
time_month            0
time_week             0
time_day              0
time_hour             0
dtype: int64
station_id            0
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
time_year             0
time_month            0
time_week             0
time_day              0
time_hour             0
dtype: int64
station_id            0
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
time_year             0
time_month            0
time_week             0
time_day              0
time_hour             0
dtype: int64
station_id            0
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentration     0
time_year             0
time_month            0
time_week             0
time_day              0
time_hour             0
dtype: int64
station_id            0
PM25_Concentration    0
PM10_Concentration    0
NO2_Concentr